In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Conv2DTranspose,UpSampling2D,MaxPooling2D,Concatenate,BatchNormalization,Flatten
from tensorflow.keras.layers import Dense,Layer
import numpy as np
from losses import *
from nets.dep_network import depth_estimation
from nets.pose_network import pose_est
from utils import *

In [2]:
#pe = pose_est()
def pose_est_loss(rot_right,rot_left,t_left,t_right):
    ##Spatial loss
    #Pose consistency loss
    lambd_p = 0.5
    lambd_o = 0.5

    pc_loss = lambd_p*tf.reduce_mean(tf.abs(t_right-t_left))+lambd_o*tf.reduce_mean(tf.abs(rot_right-rot_left))

    #Temporal loss
    
    return pc_loss

def pose_temporal_loss(pk,pk1,translate,rotate,intrinsics,depth):
    """
    Argument :
    pk is the pixel in the current frame 
    pk1 which is p_k+1 is the next frame pixel or next image 
    """
    lambd = 0.5
    pk1_pred = projective_inverse_warp(pk, depth, translate,rotate, intrinsics)
    ssim_loss_pk1 = SSIM(pk1,pk1_pred) 
    l1_loss_pk1 = tf.reduce_mean(tf.abs(tf.subtract(pk1_pred,pk1)))
    
    loss_pk1 = lambd*ssim_loss_pk1+(1-lambd)*l1_loss_pk1
    return loss_pk1
    

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
batch=3        
@tf.function
def train_step(image_left,image_right,pk,pk1):
    with tf.GradientTape(persistent=True) as tape:
        intrinsic = tf.expand_dims(tf.constant([[1,2,3],[4,2,4],[2,1,6]],dtype=tf.float32),axis=0)
        intrinsic = tf.tile(intrinsic,[batch,1,1])

        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        dcx7_right,dep_right = de(image_right)
        #dcx7_left,dep_left = de(image_left)
        #loss_depth = dept_est_loss(dep_left,dep_right,image_left,image_right)
        #tf.print("loss_depth",loss_depth)
        ##not using pose estimation currently because of tensorflow 2.0
        t_left,rot_left = pe(image_left)
        t_right,rot_right = pe(image_right)
        loss_pose = pose_est_loss(rot_right,rot_left,t_left,t_right)
        depth = 1/dep_right[:,:,:,0]
        loss_pose_1 = pose_temporal_loss(pk,pk1,t_right,rot_right,intrinsic,depth)
        #tf.print("loss_pose",loss_pose)
        loss_pose_total = loss_pose+loss_pose_1

    #gradients_de = tape.gradient(loss_depth, de.trainable_variables)
    gradients_pe = tape.gradient(loss_pose_total, pe.trainable_variables)
    #optimizer.apply_gradients(zip(gradients_de, de.trainable_variables))
    optimizer.apply_gradients(zip(gradients_pe, pe.trainable_variables))
    return loss_pose

In [7]:
EPOCHS = 5
train_ds = []
train_ps = []
de = depth_estimation()
pe = pose_est()

for i in range(5):
    #images_left = np.random.randint(0,255,(3,256,256,3)).astype(np.float32)
    #images_right = np.random.randint(0,255,(3,256,256,3*2)).astype(np.float32)
    images_left = tf.ones((3,128,416,3*2),dtype=tf.float32)*(i)
    images_right = tf.ones((3,128,416,3*2),dtype=tf.float32)*(i+2)
    train_ds.append([images_left,images_right])
for j in range(5):
    images_k = tf.ones((3,128,416,3*2))*(j)
    images_k1 = tf.ones((3,128,416,3*2))*(j+1)
    train_ps.append([images_k,images_k1])
#images_left = tf.ones((1,256,256,3*2),dtype=tf.float32)*(0)
#images_right = tf.ones((1,256,256,3*2),dtype=tf.float32)*(1)
for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    for (images_left,images_right),(images_k,images_k1) in zip(train_ds,train_ps):
        ld = train_step(images_left,images_right,images_k,images_k1)
    print("epochs : ",epoch)
    print("loss depth",ld.numpy())
    #print("loss pose",lp)

epochs :  0
loss depth 0.0005899338
epochs :  1
loss depth 0.00084391364
epochs :  2
loss depth 4.2622523e-05
epochs :  3
loss depth 2.002592e-05
epochs :  4
loss depth 1.5595773e-05


In [ ]:
batch=3
intrinsic = tf.expand_dims(tf.constant([[1,1,1],[1,1,1],[1,1,1]],dtype=tf.float32),axis=0)
intrinsic = tf.tile(intrinsic,[batch,1,1])
#intrinsic.dtype
tf.linalg.inv(intrinsic)

In [ ]:
print(inv(intrinsic)